# Train with custom container

In [3]:
import json
import os
import pandas as pd
import time

## Create a training image

### Configure Cloud Build to use Kaniko

In [13]:
!gcloud config set builds/use_kaniko True

Updated property [builds/use_kaniko].


### Create a docker file

In [15]:
APP_FOLDER = '../training_image'
os.makedirs(APP_FOLDER, exist_ok=True)

In [16]:
%%writefile $APP_FOLDER/Dockerfile

FROM gcr.io/jk-demo1/sklearn-cpu:latest
WORKDIR /app
COPY train.py .

ENTRYPOINT ["python", "train.py"]

Writing ../training_image/Dockerfile


### Create a training script

In [23]:
%%writefile $APP_FOLDER/train.py

import logging
import os
import subprocess
import sys
import joblib
import fire
import numpy as np
import pandas as pd

from sklearn.cross_decomposition import PLSRegression
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.linear_model import Ridge
from sklearn.manifold import TSNE 
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler


def train(job_dir, data_path, n_features_options, l2_reg_options):
    
  # Load data from GCS
  df_train = pd.read_csv(data_path)

  y = df_train.octane
  X = df_train.drop('octane', axis=1)
    
  pipeline = Pipeline([
    ('scale', StandardScaler()),
    ('reduce_dim', PCA()),
    ('regress', Ridge())
  ])

  param_grid = [
    {
      'reduce_dim__n_components': n_features_options,
      'regress__alpha': l2_reg_options
    }
  ]

  grid = GridSearchCV(pipeline, cv=10, n_jobs=None, param_grid=param_grid, scoring='neg_mean_squared_error', iid=False)
  
  grid.fit(X, y)

  logging.info("Best estimator: {}".format(grid.best_params_))
  logging.info("Best score: {}".format(grid.best_score_))
    
  # Retrain the best model on a full dataset
  best_estimator = grid.best_estimator_
  trained_pipeline = best_estimator.fit(X, y)

  # Save the model
  model_filename = 'model.joblib'
  joblib.dump(value=trained_pipeline, filename=model_filename)
  gcs_model_path = "{}/trained_model/{}".format(job_dir, model_filename)
  subprocess.check_call(['gsutil', 'cp', model_filename, gcs_model_path], stderr=sys.stdout)
  logging.info("Saved model in: {}".format(gcs_model_path)) 
    
if __name__ == "__main__":
  logging.basicConfig(level=logging.INFO)
  fire.Fire(train)

Overwriting ../training_image/train.py


### Build the image

In [24]:
PROJECT_ID = !gcloud config list project --format "value(core.project)"
PROJECT_ID = PROJECT_ID[0]
IMAGE_REPO_NAME="octane-regression-training"
IMAGE_TAG="latest"
IMAGE_URI="gcr.io/{}/{}:{}".format(PROJECT_ID, IMAGE_REPO_NAME, IMAGE_TAG)

!gcloud builds submit --tag $IMAGE_URI $APP_FOLDER

Creating temporary tarball archive of 2 file(s) totalling 1.9 KiB before compression.
Uploading tarball of [../training_image] to [gs://jk-demo1_cloudbuild/source/1566277759.56-59fc18c560ec44b581142f622367be5e.tgz]
Created [https://cloudbuild.googleapis.com/v1/projects/jk-demo1/builds/131ff15e-af8a-48ba-880b-ec6883624b94].
Logs are available at [https://console.cloud.google.com/gcr/builds/131ff15e-af8a-48ba-880b-ec6883624b94?project=826865698127].
----------------------------- REMOTE BUILD OUTPUT ------------------------------
starting build "131ff15e-af8a-48ba-880b-ec6883624b94"

FETCHSOURCE
Fetching storage object: gs://jk-demo1_cloudbuild/source/1566277759.56-59fc18c560ec44b581142f622367be5e.tgz#1566277759975708
Copying gs://jk-demo1_cloudbuild/source/1566277759.56-59fc18c560ec44b581142f622367be5e.tgz#1566277759975708...
/ [1 files][  1.0 KiB/  1.0 KiB]                                                
Operation completed over 1 objects/1.0 KiB.                                      
B

## Submit a training job

In [25]:
JOB_NAME="JOB_{}".format(time.strftime("%Y%m%d_%H%M%S"))
REGION="us-west1"
SCALE_TIER="BASIC"
JOB_DIR="gs://jk-demo-jobdir/{}".format(JOB_NAME)

DATA_PATH="gs://jk-demo-datasets/gasdata/training.csv"
N_FEATURES_OPTIONS="[2,4,6]"
L2_REG_OPTIONS="[0.1,0.2,0.3,0.5]"

!gcloud ai-platform jobs submit training $JOB_NAME \
--region $REGION \
--job-dir $JOB_DIR \
--master-image-uri $IMAGE_URI \
--scale-tier $SCALE_TIER \
-- \
--data_path $DATA_PATH \
--n_features_options $N_FEATURES_OPTIONS \
--l2_reg_options $L2_REG_OPTIONS

Job [JOB_20190820_050954] submitted successfully.
Your job is still active. You may view the status of your job with the command

  $ gcloud ai-platform jobs describe JOB_20190820_050954

or continue streaming the logs with the command

  $ gcloud ai-platform jobs stream-logs JOB_20190820_050954
jobId: JOB_20190820_050954
state: QUEUED


In [ ]:
!gcloud ai-platform jobs describe $JOB_NAME
!gcloud ai-platform jobs stream-logs $JOB_NAME

createTime: '2019-08-20T05:09:55Z'
etag: J8fRwIuXI2I=
jobId: JOB_20190820_050954
state: PREPARING
trainingInput:
  args:
  - --data_path
  - gs://jk-demo-datasets/gasdata/training.csv
  - --n_features_options
  - '[2,4,6]'
  - --l2_reg_options
  - '[0.1,0.2,0.3,0.5]'
  jobDir: gs://jk-demo-jobdir/JOB_20190820_050954
  masterConfig:
    imageUri: gcr.io/jk-demo1/octane-regression-training:latest
  region: us-west1
trainingOutput: {}

View job in the Cloud Console at:
https://console.cloud.google.com/mlengine/jobs/JOB_20190820_050954?project=jk-demo1

View logs at:
https://console.cloud.google.com/logs?resource=ml.googleapis.com%2Fjob_id%2FJOB_20190820_050954&project=jk-demo1
INFO	2019-08-20 05:09:55 +0000	service		Validating job requirements...
INFO	2019-08-20 05:09:55 +0000	service		Job creation request has been successfully validated.
INFO	2019-08-20 05:09:56 +0000	service		Job JOB_20190820_050954 is queued.
INFO	2019-08-20 05:09:56 +0000	service		Waiting for job to be provisioned.
IN

## Deploy using AI Platform Prediction

### Create a model resource

In [9]:
MODEL_NAME = "gas_octane_regression"
REGION = "us-central1"
LABELS = "task=regression,domain=chemometrics"

!gcloud ai-platform models create  $MODEL_NAME \
--regions=$REGION \
--labels=$LABELS

Created ml engine model [projects/jk-demo1/models/gas_octane_regression].


### Create a model version

In [22]:
MODEL_PATH = "{}/trained_model/".format(JOB_DIR)
VERSION_NAME = "octane_v02"

!gcloud ai-platform versions create $VERSION_NAME \
--model=$MODEL_NAME \
--origin=$MODEL_PATH \
--runtime-version=1.14 \
--framework=scikit-learn \
--python-version=3.5

Creating version (this might take a few minutes)......failed.                  
ERROR: (gcloud.ai-platform.versions.create) Create Version failed. Bad model detected with error:  "Failed to load model: Could not load the model: /tmp/model/0001/model.joblib. 204. (Error code: 0)"


### Test the model
#### Prepare a file with test instances

In [11]:
TEST_DATASET_PATH = "gs://jk-demo-datasets/gasdata/testing.csv"
INPUT_FILE = 'instances.json'

df_test = pd.read_csv(TEST_DATASET_PATH).drop('octane', axis=1)
with open(INPUT_FILE, "w") as f:
  for index, row in df_test.iterrows():
    f.write(json.dumps(list(row.values)))
    f.write("\n")

#### Run predictions

In [12]:
!gcloud ai-platform predict \
--model $MODEL_NAME \
--version $VERSION_NAME \
--json-instances $INPUT_FILE

[83.61274117546486, 88.78144180442887, 86.60147562277632, 86.37821707761988, 84.57226241557935, 88.60871164935777]
